# AzureML Optimizer and Profiler
This notebooks shows steps for creating optimizer and profiler jobs on the azureml platform

Install Azure-Cli  
  
- Please install azure-cli with the following command:  
`curl -sL https://aka.ms/InstallAzureCLIDeb | bash && az version`
- Or if you already have azure-cli installed, please upgrade it with the following command:  
`az upgrade -y`

In [ ]:
# install azure-cli ml extension
!az extension remove --name azure-cli-ml
!az extension remove --name ml
!az extension add --name ml -y
!az extension show --name ml

In [ ]:
import os

# Env setting for the workspace that runs the profiling job
subscription_id = os.getenv("SUBSCRIPTION_ID", default="636d700c-4412-48fa-84be-452ac03d34a1")
resource_group = os.getenv("RESOURCE_GROUP", default="model-profiler")
workspace_name = os.getenv("WORKSPACE_NAME", default="profilervalidation")

In [ ]:
# !az login
!az account set --subscription $subscription_id
!az configure --defaults group=$resource_group workspace=$workspace_name

In [ ]:
from datetime import datetime
timestamp = int(datetime.now().timestamp())

# Prepare Model

In [ ]:
%%bash

mkdir -p distilbert_model/model
python -m pip install --upgrade pip
python -m pip install --upgrade transformers[torch]

In [ ]:
from transformers import AutoModelForQuestionAnswering
import torch

model_checkpoint = "distilbert-base-cased-distilled-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

dynamic_axes = {
    "input_ids": {0: "batch_size", 1: "seq_length"},
    "attention_mask": {0: "batch_size", 1: "seq_length"},
}
input_tensor = (
    torch.ones(1, 128, dtype=torch.int64).to("cpu"),
    torch.ones(1, 128, dtype=torch.int64).to("cpu"),
)
torch.onnx.export(
    model, 
    input_tensor, 
    "distilbert_model/model/distilbert-base-cased-distilled-squad.onnx", 
    input_names=["input_ids", "attention_mask"], 
    output_names=["start_logits", "end_logits"],
    opset_version=17,
    dynamic_axes=dynamic_axes)

# Step 1: Optimize your model

In [ ]:
%cd optimizer

### Set Environment Variables

In [ ]:
%set_env OPTIMIZER_JOB_NAME=optimization-job-$timestamp
%set_env OPTIMIZER_COMPUTE_NAME=optimizerF8
%set_env INFERENCE_SERVICE_COMPUTE_SIZE=Standard_F8s_v2
%set_env OPTIMIZER_DOWNLOAD_FOLDER=../downloads/optimizer_output


### Create an Aml Compute

In [ ]:
!az ml compute create --name $OPTIMIZER_COMPUTE_NAME --size $INFERENCE_SERVICE_COMPUTE_SIZE --identity-type SystemAssigned --type amlcompute

### Create the Olive Optimizer Job Yaml

In [ ]:
%%bash

envsubst '$OPTIMIZER_COMPUTE_NAME $OPTIMIZER_JOB_NAME' < optimizer_job_template.yml > optimizer_job.yml

### Create the Olive Optimizer Job and Wait Until Job To Finish

In [ ]:
%%bash

az ml job create --name $OPTIMIZER_JOB_NAME --file optimizer_job.yml

status=""
while [[ ! "$status" =~ ^(Completed|Failed|Canceled|NotResponding|Paused)$ ]];
do
    status=$(az ml job show --name $OPTIMIZER_JOB_NAME --query "status" -o tsv)
    echo "Current status: $status"
    sleep 3
done

if [ "$status" != "Completed" ]; then echo "Optimizer job $OPTIMIZER_JOB_NAME failed!" && exit 1; fi

### Download Output Files and Copy Files to The Deployer Folder

In [ ]:
%%bash

az ml job download --name $OPTIMIZER_JOB_NAME --all --download-path $OPTIMIZER_DOWNLOAD_FOLDER

mkdir -p ../distilbert_model/optimized_model
cp $OPTIMIZER_DOWNLOAD_FOLDER/named-outputs/optimized_model/cpu-cpu_BestCandidateModel_1_model.onnx ../distilbert_model/optimized_model/distilbert-base-cased-distilled-squad.onnx
cp $OPTIMIZER_DOWNLOAD_FOLDER/named-outputs/optimized_parameters/cpu-cpu_BestCandidateModel_1_inference_config.json ../deployer/optimized_parameters.json

# Step 2: Deploy model to an oline-endpoint

In [ ]:
%cd ../deployer

### Set Environment Variables

In [ ]:
%set_env DEPLOYER_JOB_NAME=deployer-job-$timestamp
%set_env DEPLOYER_COMPUTE_NAME=deploymentTest
%set_env DEPLOYER_COMPUTE_SIZE=Standard_F4s_v2
%set_env DEPLOYER_DOWNLOAD_FOLDER=../downloads/deployer_output
%set_env ENDPOINT_NAME=distilbert-optimized-endpt
%set_env DEPLOYMENT_NAME=distilbert-optimized-dep

### Create an Aml Compute

In [ ]:
%%bash

az ml compute create --name $DEPLOYER_COMPUTE_NAME --size $DEPLOYER_COMPUTE_SIZE --identity-type SystemAssigned --type amlcompute

# Create a role-assignment
compute_info=`az ml compute show --name $DEPLOYER_COMPUTE_NAME --query '{"id": id, "identity_object_id": identity.principal_id}' -o json`
workspace_resource_id=`echo $compute_info | jq -r '.id' | sed 's/\(.*\)\/computes\/.*/\1/'`
identity_object_id=`echo $compute_info | jq -r '.identity_object_id'`
az role assignment create --role Contributor --assignee-object-id $identity_object_id --assignee-principal-type ServicePrincipal --scope $workspace_resource_id
if [[ $? -ne 0 ]]; then echo "Failed to create role assignment for compute $DEPLOYER_COMPUTE_NAME" && exit 1; fi

### Create the Deployer Job Yaml, Online-Endpoint Yaml and Online-Deployment Yaml

In [ ]:
%%bash

envsubst '$DEPLOYER_COMPUTE_NAME $DEPLOYER_JOB_NAME $ENDPOINT_NAME $DEPLOYMENT_NAME' < deployer_job_template.yml > deployer_job.yml
envsubst '$ENDPOINT_NAME $DEPLOYMENT_NAME' < endpoint_template.yml > endpoint.yml
envsubst '$ENDPOINT_NAME $DEPLOYMENT_NAME $INFERENCE_SERVICE_COMPUTE_SIZE' < deployment_template.yml > deployment.yml

### Create the Deployer Job and Wait For Job To Finish

In [ ]:
%%bash

az ml job create --name $DEPLOYER_JOB_NAME --file deployer_job.yml

status=""
while [[ ! "$status" =~ ^(Completed|Failed|Canceled|NotResponding|Paused)$ ]];
do
    status=$(az ml job show --name $DEPLOYER_JOB_NAME --query "status" -o tsv)
    echo "Current status: $status"
    sleep 3
done

if [ "$status" != "Completed" ]; then echo "Deployer job $DEPLOYER_JOB_NAME failed!" && exit 1; fi

### Download Output Files and Copy Files to The Profiler Folder and The Deleter Folder

In [ ]:
%%bash

az ml job download --name $DEPLOYER_JOB_NAME --all --download-path $DEPLOYER_DOWNLOAD_FOLDER
cp $DEPLOYER_DOWNLOAD_FOLDER/artifacts/outputs/deployment_settings.json ../profiler/
cp $DEPLOYER_DOWNLOAD_FOLDER/artifacts/outputs/deployment_settings.json ../deleter/

# Step 3: Profile your online-endpoint

In [ ]:
%cd ../profiler

### Set Environment Variables

In [ ]:
%set_env PROFILER_JOB_NAME=profiler-job-$timestamp
%set_env PROFILER_COMPUTE_NAME=profilingTest
%set_env PROFILER_COMPUTE_SIZE=Standard_F4s_v2
%set_env PROFILER_DOWNLOAD_FOLDER=../downloads/profiler_output

### Create an Aml Compute

In [ ]:
%%bash
az ml compute create --name $PROFILER_COMPUTE_NAME --size $PROFILER_COMPUTE_SIZE --identity-type SystemAssigned --type amlcompute

# Create a role-assignment
compute_info=`az ml compute show --name $PROFILER_COMPUTE_NAME --query '{"id": id, "identity_object_id": identity.principal_id}' -o json`
workspace_resource_id=`echo $compute_info | jq -r '.id' | sed 's/\(.*\)\/computes\/.*/\1/'`
identity_object_id=`echo $compute_info | jq -r '.identity_object_id'`
az role assignment create --role Contributor --assignee-object-id $identity_object_id --assignee-principal-type ServicePrincipal --scope $workspace_resource_id
if [[ $? -ne 0 ]]; then echo "Failed to create role assignment for compute $PROFILER_COMPUTE_NAME" && exit 1; fi

### Create the Profiler Job Yaml

In [ ]:
%%bash

envsubst '$PROFILER_COMPUTE_NAME $PROFILER_JOB_NAME $DEPLOYMENT_NAME' < profiler_job_template.yml > profiler_job.yml

### Create the Profiler Job and Wait For Job To Finish

In [ ]:
%%bash

az ml job create --name $PROFILER_JOB_NAME --file profiler_job.yml

status=""
while [[ ! "$status" =~ ^(Completed|Failed|Canceled|NotResponding|Paused)$ ]];
do
    status=$(az ml job show --name $PROFILER_JOB_NAME --query "status" -o tsv)
    echo "Current status: $status"
    sleep 3
done

if [ "$status" != "Completed" ]; then echo "Profiler job $PROFILER_JOB_NAME failed!" && exit 1; fi

### Download Output Files

In [ ]:
!az ml job download --name $PROFILER_JOB_NAME --all --download-path $PROFILER_DOWNLOAD_FOLDER

# Step 4: Delete the online-endpoint

In [ ]:
%cd ../deleter

### Set Environment Variables

In [ ]:
%set_env DELETER_JOB_NAME=deleter-job-$timestamp
# will reuse the aml compute for the online-endpoint deployer job
%set_env DELETER_COMPUTE_NAME=deploymentTest
%set_env DELETER_DOWNLOAD_FOLDER=../downloads/deleter_output

### Create the Deleter Job Yaml

In [ ]:
%%bash

envsubst '$DELETER_COMPUTE_NAME $DELETER_JOB_NAME $ENDPOINT_NAME $DEPLOYMENT_NAME' < deleter_job_template.yml > deleter_job.yml

### Create the Deleter Job and Wait For Job To Finish

In [ ]:
%%bash

az ml job create --name $DELETER_JOB_NAME --file deleter_job.yml

status=""
while [[ ! "$status" =~ ^(Completed|Failed|Canceled|NotResponding|Paused)$ ]];
do
    status=$(az ml job show --name $DELETER_JOB_NAME --query "status" -o tsv)
    echo "Current status: $status"
    sleep 3
done

if [ "$status" != "Completed" ]; then echo "Deleter job $DELETER_JOB_NAME failed!" && exit 1; fi

### Download Output Files

In [ ]:
!az ml job download --name $DELETER_JOB_NAME --all --download-path $DELETER_DOWNLOAD_FOLDER